In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_football/tensors/"
tensor_dir = "./test_tensors/layer8/"
output_dir = "./measurements/"
layer = 8

test_tensors = os.listdir(tensor_dir)

In [3]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [4]:
for i in range(10):
    print("************* Testing tensor: "+str(i)+"***************")
    split_layer.append(layer)
    tensor1 = torch.load(tensor_dir+test_tensors[i])
    tensor2 = torch.load(tensor_dir+test_tensors[i+1])
    tensor = tensor2 - tensor1
    tensor = tensor * (abs(tensor)>0.01)
    tensor = tensor[0]

     # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(1-get_tensor_pictoriality(tensor)/(tensor.numel()*4))
    tensor_regularity.append(1-get_tensor_regularity(tensor)/(tensor.numel()*4))
    tensor_decomposability.append(1-((tensor_rank[-1]*tensor.shape[1]+tensor_rank[-1]*tensor.shape[2])*tensor.shape[0])/(tensor.numel()*4))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    jpeg_ratio.append(ratio_jpeg)
    jpeg_snr.append(snr_jpeg)
    jpeg_mse.append(mse_jpeg)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    decomposition_ratio.append(ratio_decomposition)
    decomposition_snr.append(snr_decomposition)
    decomposition_mse.append(mse_decomposition)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    regression_ratio.append(ratio_regression)
    regression_snr.append(snr_regression)
    regression_mse.append(mse_regression)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    sketchml_ratio.append(ratio_sketchml)
    sketchml_snr.append(snr_sketchml)
    sketchml_mse.append(mse_sketchml)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


************* Testing tensor: 0***************
Rank: 17.851562, Sparsity: 0.813193, Picoriality: 0.932078, Regularity: 0.990802, Decomposability: 0.656701
Doing JPEG compression
Ratio: 6.196062, SNR: 29.966273, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.728228, SNR: 135.219021, MSE: 0.000000
Doing Regression compression
Ratio: 16.628009, SNR: 7.735867, MSE: 0.000021
Doing SketchML compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/sketchml.py:185: RuntimeWarning: Mean of empty slice.
  bucket_means[k] = values[bucket_mask].mean()
/home/rex/archiconda3/envs/pytorch/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Ratio: 19.617525, SNR: 1.183305, MSE: 0.000093
************* Testing tensor: 1***************
Rank: 17.710938, Sparsity: 0.855319, Picoriality: 0.944861, Regularity: 0.994798, Decomposability: 0.659405
Doing JPEG compression
Ratio: 5.787439, SNR: 29.226568, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.734010, SNR: 135.559425, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 7.098190, MSE: 0.000018
Doing SketchML compression
Ratio: 24.070659, SNR: 1.222394, MSE: 0.000071
************* Testing tensor: 2***************


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/sketchml.py:185: RuntimeWarning: Mean of empty slice.
  bucket_means[k] = values[bucket_mask].mean()
/home/rex/archiconda3/envs/pytorch/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Rank: 17.460938, Sparsity: 0.853885, Picoriality: 0.948384, Regularity: 0.996358, Decomposability: 0.664213
Doing JPEG compression
Ratio: 6.272531, SNR: 28.224385, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.744519, SNR: 135.309248, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 6.972793, MSE: 0.000020
Doing SketchML compression
Ratio: 23.920934, SNR: 1.283505, MSE: 0.000076
************* Testing tensor: 3***************
Rank: 17.640625, Sparsity: 0.859537, Picoriality: 0.949545, Regularity: 0.996708, Decomposability: 0.660757
Doing JPEG compression
Ratio: 6.248299, SNR: 27.668681, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.736935, SNR: 135.554781, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 6.894786, MSE: 0.000019
Doing SketchML compression
Ratio: 24.683497, SNR: 1.317379, MSE: 0.000069
************* Testing tensor: 4***************
Rank: 17.820312, Sparsity: 0.836804, Picoriality: 0.952294, Regularity: 0.997255, Decompos

/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Rank: 19.523438, Sparsity: 0.774316, Picoriality: 0.925289, Regularity: 0.991748, Decomposability: 0.624549
Doing JPEG compression
Ratio: 5.209784, SNR: 31.657751, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.665866, SNR: 135.015955, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 7.393275, MSE: 0.000031
Doing SketchML compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Ratio: 16.734939, SNR: 1.202150, MSE: 0.000131
************* Testing tensor: 9***************
Rank: 19.773438, Sparsity: 0.774027, Picoriality: 0.924201, Regularity: 0.991338, Decomposability: 0.619742
Doing JPEG compression
Ratio: 5.286655, SNR: 31.003358, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.657448, SNR: 135.097208, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 7.312987, MSE: 0.000032
Doing SketchML compression
Ratio: 16.737366, SNR: 1.178684, MSE: 0.000130


In [5]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [6]:
df.to_csv(output_dir+"layer"+str(layer)+"_diff.csv",index=False, header=True)